In [1]:
import sys
sys.path.append('./utils/')
import pandas as pd
import numpy as np
import os 
import cv2
from skimage.io import imread,imshow
from utils_fcn import DataGeneratorFCN
import matplotlib.pyplot as plt
import keras.callbacks
%matplotlib inline

/home/wvillegas/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k
 
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))
###################################

In [3]:
dataset = pd.read_csv(os.path.join('/home','wvillegas','dataset-mask', 'full_masks.csv'))

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset['orig'],dataset['mask'],
                                                    test_size = 0.2, random_state=1)

In [6]:
partition = {'train':list(X_train),
             'test': list(X_test)}

In [7]:
img_list = list(X_train) + list(X_test)
mask_list = list(Y_train) + list(Y_test)

In [8]:
labels = dict(zip(img_list, mask_list))

In [9]:
img_path = os.path.join('/home','wvillegas','dataset-mask','dataset_resize', 'images_resize')
masks_path = os.path.join('/home','wvillegas','dataset-mask','dataset_resize', 'masks_resize')

In [10]:
batch_size = 1
# dim_img = (1728, 2304)
# dim_mask = (658,874)

In [11]:
train_generator = DataGeneratorFCN(batch_size=batch_size,img_path=img_path,
                                labels=labels,list_IDs=partition['train'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)
valid_generator = DataGeneratorFCN(batch_size=batch_size,img_path=img_path,
                                labels=labels,list_IDs=partition['test'],n_channels=3,
                                n_channels_label=1,shuffle=True,mask_path=masks_path)

In [12]:
from keras.applications import VGG16
from keras.layers import Conv2DTranspose,Conv2D, Add, Cropping2D
from keras import Model
from keras.models import load_model

In [13]:
net = VGG16(include_top=False, weights='imagenet')
# net.load_weights('/home/wvillegas/DLProjects/BudClassifier/cmdscripts/modelosV2/mobilenet_weights_detection.h5', by_name=True)

In [14]:
# for layer in net.layers:
#     layer.trainable = True

In [15]:
# test arquitectura paper de FCN 
# deconv1 = Conv2D(filters=256,kernel_size=3,strides=1,activation='relu')(mobilenet.output)
predict = Conv2D(filters=1,kernel_size=1,strides=1)(net.output)
deconv1 = Conv2DTranspose(filters=1,kernel_size=64,strides=32, padding='same')(predict)

In [16]:
fcn = Model(inputs=net.input,outputs=deconv1)

In [17]:
from keras.optimizers import SGD, RMSprop, Adam

In [18]:
sgd = SGD(lr=0.0001,momentum=0.9,decay=0.0005)
fcn.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [19]:
history = fcn.fit_generator(generator=train_generator,validation_data=valid_generator,
                            use_multiprocessing=True,workers=6, epochs=1)

Epoch 1/1


InternalError: Dst tensor is not initialized.
	 [[Node: _arg_input_1_0_0/_237 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_1141__arg_input_1_0_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: loss/mul/_261 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1153_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
list_img = partition['test'][2:4]

In [ ]:
inp_img = []
for img in list_img:
    temp = cv2.imread(img_path + '/' + img)
    temp = cv2.normalize(temp, temp, 0, 255, cv2.NORM_MINMAX)
    inp_img.append(temp)
    
ground_truth = []
for img in list_img:
    mask = labels[img]
    temp = cv2.imread(masks_path + '/' + mask)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    temp = temp.astype(bool).astype(int)
    ground_truth.append(temp)


In [ ]:
inp_img = np.asarray(inp_img)
ground_truth = np.asarray(ground_truth)

In [ ]:
inp_img.shape

In [ ]:
imshow(inp_img[0,:,:,0])

In [ ]:
pred = fcn.predict(inp_img)

In [ ]:
# img = cv2.imread(img_path + '/0717.jpg')
# img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
# img.shape

In [ ]:
# xx = np.empty((1,img.shape[0],img.shape[1], 3))
# xx[0,] = img

In [ ]:
# pred = fcn.predict(xx)
# pred.shape

In [ ]:
pred.shape

In [ ]:
imshow(pred[0,:,:,0])

In [ ]:
ground_truth.shape

In [ ]:
imshow(ground_truth[0,:,:])

In [ ]:
ground_truth[0,:,:,0].sum()